### 스팸 게시글 구분을 위한 SVM 만들기
* 스팸 게시글을 구분하기 위한 SVM 구축을 목적으로 합니다.
* 기본적인 SVM 구축을 위한 데이터 전처리 및 단어 벡터화의 내용을 다룹니다.

In [1]:
import pandas as pd
import numpy as np
import nltk as nlt
import os
import re
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
train_df = pd.read_csv('./dataset/train/train.csv', index_col = 0)
train_X, train_y = train_df['title'].str.lower(), train_df['label']

In [3]:
train_X

0        bitcoin mvrv drops below  as investors deposit...
1        change your life lpn token register now. https...
2        saw this upsetting comment on a financial depr...
3        for guaranteed investment and resolve of compl...
4                 crypto tax calculator | cryptotrader.tax
                               ...                        
23608    dumb question if people cant afford to buy a f...
23609    miss the last bull run dont dismay relive all ...
23610    how to profit from the  cryptocurrency bullrun...
23611    bitcoin market insights webinar - bitcoin indu...
23612                           billion market cap reached
Name: title, Length: 23613, dtype: object

#### 특수 문자 및 이모지를 텍스트로 치환해준다.
* url의 경우 httpadr로 변환한다.
* 이모지의 경우 emot를 사용해 텍스트로 변환한다.
* $나 !의 경우 각각 dollar 와 exclamation으로 변환해준다.

In [4]:
def replace_regex(title_series):
    url_regex = "(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}"
    title_series = title_series.str.replace(url_regex, 'httpaddr',regex=True)
    title_series = title_series.str.replace('[$]+','dollar',regex=True)
    title_series = title_series.str.replace('[!]+','excalmation',regex=True)
    title_series = title_series.str.replace('[0-9]+','number',regex=True)
    return title_series

In [5]:
##이모티콘을 텍스트로 치환해준다.
def replace_emoji(title_series): #토큰화를 진행한 후 각 토큰에 존재하는 이모지들을 텍스트로 치환해준다.
    import emot
    emot_obj = emot.core.emot()
    for v in title_series:
        emoji = emot_obj.emoji(v)
        if emoji['value']:
            v += ' '.join(emoji['value'])
    title_series = title_series.str.replace('[^a-zA-Z ]','',regex=True) #불용어 전부 제거
    return title_series

In [6]:
train_X = replace_regex(train_X)
train_X = replace_emoji(train_X)

#### 불필요한 기호나 부호 이모지를 제거해준다.
* 이모지 제거
* 특수기호 제거
* 숫자 제거

##### 토큰화와 표제어 추출
* 앞서 우리는 토큰화에 앞서 처리해야 하는 특수 단어들을 대체 했다.
* 이제 문장들을 토큰 단위로 쪼개고 하나의 표제어로 묶는 작업을 진행해 보자.


In [7]:
def tokenize(title_sereis):
    tokenizer = nlt.tokenize.TreebankWordTokenizer()
    tokenized_data = title_sereis.apply(tokenizer.tokenize) #트리뱅크 토크나이저로 문장을 토큰화 해준다.
    return tokenized_data

In [8]:
def pos_tag(sentences):
    #표제어 추출에 앞서 각 토큰에 품사를 결정해준다.
    #품사를 미리 지정해 줌으로써 표제어 추출의 정확도를 향상 시킬 수 있다.
    pos_list = [nlt.pos_tag(token) for token in sentences]
    return pos_list

In [ ]:
pos_list = pos_tag(tokenize(train_X)) #품사가 지정된 토큰 리스트를 반환 받는다.
pos_list

In [10]:
def lemmatize(pos_list):
    from nltk.corpus import stopwords
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer() #표제어 추출기 
    lemmatized_res = []
    for sentece in pos_list:
        temp = [] #한 문장에서 추출된 표제어를 임시로 저장한다.
        for token, pos in sentece:
            #lemmatize 함수의 매개변수는 v,a,n,r로 동 형 명 부 사 밖에 없기 때문에 다시금 매핑을 진행해야 한다.
            if token not in stopwords.words('english') and pos[0] in ['V','J','N','R']: 
                #불용어가 아니고 동,형,명,부사의 한 종류일 경우 표제어 추출을 진행한다.
                #j는 a로 취급해줘야 한다. lemmtizer는 형용사를 pos와 다른 알파벳으로 취급한다.
                _pos = 'a' if pos[0] == 'J' else pos.lower()[0] 
                temp.append((token,_pos))
        lemmatized_res.append([lemmatizer.lemmatize(t,p) for t,p in temp]) #표제어를 추출한다.
    return lemmatized_res

In [ ]:
lemmatized_tokens = lemmatize(pos_list) #표제어가 추출된 토큰 리스트
lemmatized_tokens[:10]

In [12]:
lemma_df = pd.DataFrame([lemmatized_tokens, train_y]).T
lemma_df.columns = ['tokens','label'] 
lemma_df

,tokens,label
0,"[bitcoin, mvrv, drop, investor, deposit, b, bt...",0
1,"[change, life, lpn, token, register, httpaddrr...",1
2,"[saw, upsetting, comment, financial, depressio...",0
3,"[guarantee, investment, resolve, complaint, co...",1
4,"[crypto, tax, calculator, httpaddr]",1
...,...,...
23608,"[dumb, question, people, cant, afford, buy, fu...",0
23609,"[miss, last, bull, run, dont, dismay, relive, ...",0
23610,"[profit, cryptocurrency, bullrun, potentially,...",0
23611,"[bitcoin, market, insight, webinar, bitcoin, i...",1


#### 표제어 추출의 결과
* 단어를 토큰화하고 표제어 추출을 진행하니 문장의 원초적인 요소들만이 남은 것을 확인 할 수 있다.
* 이제 불용어 및 빈도수가 낮은 단어들을 처내고 사용할 단어들을 선별하는 작업을 진행해보자

#### 잠깐! 이런 단어들은 어쩌지?
* 우리는 게시글 데이터를 가격과 연관이 있는 게시글 그리고 가격과 연관이 없는 게시글로 분류를 하고 있다.
* 이때 두 종류의 글에서 동시에 자주 등장하는 단어의 경우 어떻게 처리해야 할까?

In [13]:
def create_frequent_sereis(lemmatized_tokens, freq_bound = 50, word_length = 2):
    #각 단어의 빈도수를 추출한다.
    word_freq = pd.Series(np.concatenate([w for w in lemmatized_tokens])).value_counts()
    words_available = word_freq.loc[(word_freq > freq_bound)] #10회 이상 사용된 단어로만
    indices = pd.Series(words_available.index)
    #단어의 길이가 2 이상인 단어들로
    indices = indices.loc[indices.apply(len) > word_length]
    freq_series = words_available[indices] #사용할 단어들만을 모아 단어,빈도수 딕셔너리를 생성하자
    return freq_series

In [14]:
def get_doubled_words(lemma_df):
    neg_df = lemma_df.groupby('label').get_group(0)
    pos_df = lemma_df.groupby('label').get_group(1)
    pos_lemma, neg_lemma = pos_df['tokens'], neg_df['tokens']
    pos_freq = create_frequent_sereis(pos_lemma)
    neg_freq = create_frequent_sereis(neg_lemma)
    doubled_words = [i for i in pos_freq.index if i in neg_freq.index]
    return doubled_words


#### 겹치는 단어들을 살펴보니
* 겹치는 단어들의 모습을 살펴보니 우리가 정말로 어떤 형식의 글에서든 존재할 수 밖에 없는 단어들이었다.
* 해당 단어들은 데이터의 성격상 늘상 존재할 수 밖에 없는 단어 이므로 특정 데이터의 성질을 판단하는데 사용할 수 없다.
* 어디에나 있는 것으로 판단할 수 없기 때문이다.

In [15]:
def remove_doubled_words(freq_series, words):
    for w in words:
        try:
            freq_series.drop(w, inplace = True)
        except Exception as e:
            continue

In [16]:
def create_rank_series(freq_series):
    freq_rank = []
    rank = 1
    before = freq_series[0]
    for f in freq_series:
        if f == before: #빈도수가 동일하면 동일한 랭크를 갖는다.
            freq_rank.append(rank)
        else:
            rank += 1
            freq_rank.append(rank)
            before = f
    return freq_rank

In [ ]:
freq_series = create_frequent_sereis(lemmatized_tokens, 300)
remove_doubled_words(freq_series, get_doubled_words(lemma_df))
freq_dict = {i:r for i,r in zip(freq_series.index, create_rank_series(freq_series))}
freq_dict

#### 단어의 벡터화
* 이제 앞서 만든 빈도수 사전을 통해 문장을 벡터로 표현해보자.
* 각 단어는 문자 대신 빈도수 형태로 표현된다.

In [18]:
def vectorize(tokens, freq_dict):
    vectorized = []
    max_length = max(map(len,tokens)) #가장 긴 문장의 단어 개수
    for sentence in tokens:
        vector = [freq_dict[word] if word in freq_dict.keys() else 0 for word in sentence]
        vector.extend([0] * (max_length - len(vector)))
        vectorized.append(vector)
    return vectorized

In [19]:
vectors = np.array(vectorize(lemmatized_tokens, freq_dict))
vectors = (vectors - vectors.mean()) / (vectors.std())
vectors

array([[-0.0889647, -0.0889647, -0.0889647, ..., -0.0889647, -0.0889647,
        -0.0889647],
       [-0.0889647, -0.0889647, -0.0889647, ..., -0.0889647, -0.0889647,
        -0.0889647],
       [-0.0889647, -0.0889647, -0.0889647, ..., -0.0889647, -0.0889647,
        -0.0889647],
       ...,
       [-0.0889647, -0.0889647, -0.0889647, ..., -0.0889647, -0.0889647,
        -0.0889647],
       [-0.0889647, -0.0889647, -0.0889647, ..., -0.0889647, -0.0889647,
        -0.0889647],
       [-0.0889647, -0.0889647, -0.0889647, ..., -0.0889647, -0.0889647,
        -0.0889647]])

#### 마침내!! 드디어!!
> 우리는 이제야 데이터 전처리 과정을 마쳤다. 긴 여정이었다. 지나온 길을 빠르게 돌아보자.
1. 결측치 제거
2. 이모티콘, 느낌표, 달러 등 유의미한 특수기호 텍스트로 변환
3. 의미 없는 이모지, 특수기호, 숫자등 알파벳이 아닌 문자 전부 제거
4. 토크나이징
5. 토큰에 품사 태깅
6. 품사 태깅된 토큰 리스트를 사용해 표제어 추출
7. 표제어 중에서 불용어 제거 후 각 단어별 빈도수 순위 사전 생성
8. 사전에 존재하는 순위를 활용해 각 표제어 토큰 리스트를 랭킹 정보를 갖고 있는 정수 벡터로 치환
9. 정수 벡터 리스트 생성

#### 이젠 모델링이 하고 싶어요..
* 선생님.. 모델링이 하고 싶어요..
* 먼길 돌아왔다. 이젠 모델링을 진행해보자.
* 우리는 SVM을 사용해 분류를 진행할 것이다.
* 학습,검증,테스트를 6:2:2 비율로 사용할 것이다.

In [20]:
#모델 평가를 위한 평가지표인 f1 스코어를 정의 해준다.
def get_f1(prediction, label_data, C):
    pos_true = label_data.loc[(prediction == 1) & (label_data == prediction)].size
    pos_false = label_data.loc[(prediction == 1) & (label_data != prediction)].size
    neg_false = label_data.loc[(prediction == 0) & (label_data != prediction)].size
    precision = pos_true / (pos_true + pos_false)
    recall = pos_true / (pos_true + neg_false)
    f1_score = 2 * (precision * recall) / (precision + recall)
    accuracy = accuracy_score(prediction, label_data) * 100
    print(f'precision:{precision}, recall:{recall}, f1:{f1_score}')
    print(f'accuracy:{accuracy}')
    return {'C': C, 'accuracy': accuracy, 'recall': recall, 'precision': precision, 'F1-score': f1_score}

In [21]:
SVM = svm.SVC(C = 0.3, kernel = 'linear', degree = 2, gamma = 'auto')
SVM.fit(vectors, train_y)
prediction = SVM.predict(vectors)
get_f1(prediction, train_y)

TypeError: get_f1() missing 1 required positional argument: 'C'

#### 정확도 증가를 위한 새로운 방향
* 정확도 자체는 나쁘지 않지만, 리콜 값이 무척이나 밑돌고 있는 점을 확인 할 수 있다.
* 이는 모델이 neg_false를 많이 도출하고 있다는 의미이며, 이는 스팸이 아닌 메일을 스팸으로 분류하는 경우가 다분하다는 뜻이다.
* 벡터라이징 기법에 문제가 있는 것은 아닐까? 
* 단순히 빈도수를 통한 순위가 아닌 조금 더 수학적인 기법을 활용해 벡터라이징을 적용해보자.

In [ ]:
def get_tfidfvec(tokens, freq_dict):
       #필터를 통해 유의미한 단어만 문장에서 걸러준다.
       tfidfv_words = [list(filter(lambda x: x in freq_dict.keys(), sentence)) for sentence in tokens]
       #해당 단어가 한개도 존재하지 않는 문장일 경우 공백으로 표시해준다.
       corpus = list(map(lambda x: ' '.join(x) if len(x) > 0 else " ", tfidfv_words))
       tfidfv = TfidfVectorizer().fit(corpus)
       return tfidfv.transform(corpus).toarray()

In [ ]:
tfidfv = get_tfidfvec(lemma_df['tokens'], freq_dict)
tfidfv.shape

(23613, 15)

In [ ]:
#tf-id 벡터라이징을 적용한 SVM
SVM = svm.SVC(C = 0.3, kernel = 'linear', degree = 2, gamma = 'auto')
SVM.fit(tfidfv, train_y)
prediction = SVM.predict(tfidfv)
get_f1(prediction, train_y)

precision:0.9954582989265071, recall:0.7942024211479864, f1:0.8835142687004718
accuracy:89.23050861813408


### TF-ID 기법이 확실히 더 나은 성능을 보인다.
* 벡터라이징도 짱구를 굴려서 작업을 해야한다.
* 자세한 사항은 링크를 통해 학습하자.
* https://wikidocs.net/31698

#### 검증셋을 통해 최적의 C와 감마 등을 탐색해보자.
* 검증 셋을 활용해 가장 결과가 괜찮은 파라미터 수치를 설정하자

In [ ]:
#앞 선 전처리 과정을 한번에 진행해주는 함수
def preprocess(X, freq_dict, tfidfv = True):
    X_data = replace_regex(X)
    X_data = replace_emoji(X_data)
    tokenized_data = tokenize(X_data)
    pos_list = pos_tag(tokenized_data)
    tokens = lemmatize(pos_list)
    #표제어 데이터 프레임 생성

    #tfid를 사용한다.
    if tfidfv:
        vectorized = get_tfidfvec(tokens,freq_dict)
    else:
        vectorized = np.array(vectorize(tokens, freq_dict))
        vectorized = (vectorized - vectorized.mean()) / (vectorized.std())
    return vectorized

In [ ]:
valid_df = pd.read_csv('./dataset/train/valid.csv',index_col = 0)
valid_X, valid_y = valid_df['title'].str.lower(), valid_df['label']
vectors = preprocess(valid_X, freq_dict)
SVM = svm.SVC(C = 0.3, kernel = 'linear', degree = 2, gamma = 'auto')
SVM.fit(vectors, valid_y)
valid_prediction = SVM.predict(vectors)
f1 = get_f1(valid_prediction, valid_y)

precision:0.9939277724512624, recall:0.784957092377587, f1:0.877168241432802
accuracy:88.93406174564859


{'accuracy': 88.93406174564859,
 'recall': 0.784957092377587,
 'precision': 0.9939277724512624,
 'F1-score': 0.877168241432802}

In [ ]:
def optimize(SVM, vectors, y):
    y_range = [0.01, 0.03, 0.1, 0.3, 1, 3]
    df = pd.DataFrame()
    for c_value in y_range:
        SVM = svm.SVC(C = c_value, kernel = 'linear', degree = 2, gamma = 'auto')
        SVM.fit(vectors, y)
        prediction = SVM.predict(vectors)
        df = df.append(get_f1(prediction, y, c_value), ignore_index = True)
    return df

In [ ]:
res = optimize(SVM, vectors, valid_y)

precision:0.993747943402435, recall:0.762241292276628, f1:0.8627338951578346
accuracy:87.79062380891881
precision:0.9939277724512624, recall:0.784957092377587, f1:0.877168241432802
accuracy:88.93406174564859
precision:0.9939277724512624, recall:0.784957092377587, f1:0.877168241432802
accuracy:88.93406174564859
precision:0.9939277724512624, recall:0.784957092377587, f1:0.877168241432802
accuracy:88.93406174564859
precision:0.9939277724512624, recall:0.784957092377587, f1:0.877168241432802
accuracy:88.93406174564859
precision:0.9939277724512624, recall:0.784957092377587, f1:0.877168241432802
accuracy:88.93406174564859


In [ ]:
res.loc[res['F1-score'].max() == res['F1-score']]

,C,F1-score,accuracy,precision,recall
1,0.03,0.877168,88.934062,0.993928,0.784957
2,0.10,0.877168,88.934062,0.993928,0.784957
3,0.30,0.877168,88.934062,0.993928,0.784957
4,1.00,0.877168,88.934062,0.993928,0.784957
5,3.00,0.877168,88.934062,0.993928,0.784957


#### 최후의 테스트 진행해보기.
* 우리는 지금까지 텍스트의 전처리 벡터라이징을 진행한 뒤 모델을 학습시켰고
* 이후 모델의 파라미터를 최적화 하기 위해 optimize 함수를 활용해 최적화된 C값을 찾아봤다.
* 이제 최적화 된 C값을 통해 모델의 최종 결과를 확인해보자.

In [116]:
test_df = pd.read_csv('./dataset/train/test.csv',index_col = 0)
test_X, test_y = test_df['title'].str.lower(), test_df['label']
vectors = preprocess(test_X, freq_dict)
SVM = svm.SVC(C = 0.03, kernel = 'linear', degree = 2, gamma = 'auto')
SVM.fit(vectors, test_y)
test_prediction = SVM.predict(vectors)
f1 = get_f1(test_prediction, test_y, 0.03)

precision:0.9965222889661713, recall:0.7868197703444832, f1:0.879341609708467
accuracy:89.01029094143057
